# Test some tensorflow 2 functionality


In [8]:
import tensorflow as tf
import numpy as np

In [16]:
@tf.function
def add(a, b):
  return a + b

number_elements = 1e5
arr1 = np.arange(0,number_elements)
arr2 = np.arange(0,number_elements)
arr3 = tf.Variable(arr1)
arr4 = tf.Variable(arr2)

out1 = add(arr3, arr4)  
tf.print(out1)

[0 2 4 ... 199994 199996 199998]


In [18]:
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Please install GPU version of TF
